In [ ]:
import pandas as pd
import json

In [ ]:
from ipywidgets import IntProgress
from IPython.display import display

### load data

In [ ]:
def load_from_telegram(telegram_file_name, user = ''):
    with open(telegram_file_name) as f:
        file_data = json.load(f)

    df = pd.DataFrame(columns=['text'])
    for msg in file_data['messages']:    
        if type(msg['text'])==type('') and len(msg['text'])>20 and len(msg['text'])<200:
            if user == '' or msg['from']==user:
                df = df.append({'text':msg['text']}, ignore_index=True)
    df = df[~df.duplicated(subset=['text'], keep=False)]
    df.to_csv('data.csv')
    return df

In [ ]:
def restore_data_from_file():
    return pd.read_csv('data.csv')

In [ ]:
#df = load_from_telegram('telegram.json','Vladimir Korotkov')
df = restore_data_from_file()
len(df)

### prepare model

In [ ]:
from deeppavlov import build_model, configs

In [ ]:
model = build_model(configs.squad.squad_ru_rubert_infer, download=False)

In [ ]:
def get_answer(df,question):
    
    # select phrases
    pre_words = question.replace("?","").split(" ")
    words =[]
    for word in pre_words:
        if len(word)>2:
            words.append(word)    
    if len(words)==0:
        return 'Слишком короткий запрос'    
    
    df = df[df.text.str.contains('|'.join(words), regex = True)]
    
    if len(words)==0:
        return 'Нет данных'
    
    data = df.text.values.tolist()
    
    prgBar = IntProgress(min = 0, max = len(data))
    display(prgBar)
    
    answers = pd.DataFrame(columns=['answer','score'])
    for theme in data:
        res = model([theme], [question])
        answers = answers.append({'answer':res[0][0], 'score':res[2][0]}, ignore_index=True)            
        prgBar.value = prgBar.value + 1
        
    max_score = answers.score.max()    
    print('answers len',len(answers))
    print('answers',answers.head())
    return answers[answers.score==max_score].iloc[0].answer

In [ ]:
answer = get_answer(df,'Нужно ли сортировать мусор?')
answer

In [ ]:
df.head()

In [5]:
text = "Как дела?"
question = str(text)
if '??' in question:
    question = question.replace('??','?')
question

'Как дела??'